In [2]:
from pyspark.sql import functions as F

# 1. Read Delta tables from Bronze layer
emails_df       = spark.read.format("delta").load("Tables/Bronze/emails")
team_messages_df = spark.read.format("delta").load("Tables/Bronze/teams_messages")
tickets_df      = spark.read.format("delta").load("Tables/Bronze/tickets")

# 2. Cast date/time columns to timestamp
emails_df       = emails_df.withColumn("received_utc", F.to_timestamp("received_utc"))
team_messages_df = team_messages_df.withColumn("timestamp_utc", F.to_timestamp("timestamp_utc"))
tickets_df      = tickets_df.withColumn("opened_utc", F.to_timestamp("opened_utc"))

# 3. Write as Delta tables in Silver layer
emails_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("Tables/Silver/Customer/email")
team_messages_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("Tables/Silver/Customer/chat")
tickets_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("Tables/Silver/Customer/tickets")

print("✅ Delta tables created in Silver layer: email, chat, tickets")

StatementMeta(, 892746de-2dc1-4971-b1c1-09e911577c90, 4, Finished, Available, Finished)

✅ Delta tables created in Silver layer: email, chat, tickets


In [3]:
# Attach Lakehouse as Default before running this cell
excel_path = "Files/Bronze/billing_data.xlsx"  # your uploaded file in Lakehouse Files

import pandas as pd
from pyspark.sql import functions as F

# -----------------------
# 1) Read Bronze (Excel) into pandas
# -----------------------
invoices_pdf  = pd.read_excel(f"/lakehouse/default/{excel_path}", sheet_name="invoices_raw")
customers_pdf = pd.read_excel(f"/lakehouse/default/{excel_path}", sheet_name="customers_raw")
products_pdf  = pd.read_excel(f"/lakehouse/default/{excel_path}", sheet_name="products_raw")

# -----------------------
# 2) Convert pandas → Spark (Bronze in memory)
# -----------------------
invoices_bronze_sdf  = spark.createDataFrame(invoices_pdf)
customers_bronze_sdf = spark.createDataFrame(customers_pdf)
products_bronze_sdf  = spark.createDataFrame(products_pdf)

# (Optional) Persist the raw as Delta under Bronze for lineage/replay
# invoices_bronze_sdf.write.format("delta").mode("overwrite").save("Tables/Bronze/invoices_raw")
# customers_bronze_sdf.write.format("delta").mode("overwrite").save("Tables/Bronze/customers_raw")
# products_bronze_sdf.write.format("delta").mode("overwrite").save("Tables/Bronze/products_raw")

# -----------------------
# 3) QUICK DEBUG: confirm input columns so selects won’t fail
# -----------------------
print("Bronze invoices columns:", invoices_bronze_sdf.columns)
print("Bronze customers columns:", customers_bronze_sdf.columns)
print("Bronze products columns:",  products_bronze_sdf.columns)

# Expected from your bronze:
# invoices_raw: invoice_id, cust_id, invoice_date, usage_period_start, usage_period_end,
#               total_amount_raw, currency, high_cost_product_code, high_cost_product_price_raw,
#               payment_status_raw, payment_method_raw, due_date, late_fee_flag_raw, discount_rate_raw
# customers_raw: cust_id, name, email, account_name
# products_raw: product_code, product_name

# -----------------------
# 4) Bronze → Silver: invoices (create columns FIRST, then select)
# -----------------------
silver_invoices = (
    invoices_bronze_sdf
    # Create typed/clean columns (do not select yet)
    .withColumn("InvoiceID",   F.col("invoice_id"))
    .withColumn("CustomerID",  F.col("cust_id"))
    .withColumn("InvoiceDate", F.to_date("invoice_date", "M/d/yyyy"))
    .withColumn("UsageStart",  F.to_date("usage_period_start", "M/d/yyyy"))
    .withColumn("UsageEnd",    F.to_date("usage_period_end", "M/d/yyyy"))
    .withColumn("TotalAmount",
                F.regexp_replace(F.col("total_amount_raw"), ",", "").cast("decimal(18,2)"))
    .withColumn("Currency",    F.col("currency"))
    .withColumn("ProductCode", F.col("high_cost_product_code"))
    .withColumn("HighCostProductPrice",
                F.regexp_replace(F.col("high_cost_product_price_raw"), ",", "").cast("decimal(18,2)"))
    .withColumn("PaymentStatus", F.col("payment_status_raw"))
    .withColumn("PaymentMethod", F.col("payment_method_raw"))
    .withColumn("DueDate",       F.to_date("due_date", "M/d/yyyy"))
    .withColumn("LateFeeApplied", F.when(F.lower(F.col("late_fee_flag_raw")) == "yes", F.lit(True)).otherwise(F.lit(False)))
    .withColumn("DiscountApplied", F.col("discount_rate_raw"))  # keep as string; format at Gold if needed
    # Now that the columns exist, select the final Silver projection
    .select(
        "InvoiceID","CustomerID","InvoiceDate","UsageStart","UsageEnd",
        "TotalAmount","Currency","ProductCode","HighCostProductPrice",
        "PaymentStatus","PaymentMethod","DueDate",
        "LateFeeApplied","DiscountApplied"
    )
)

# -----------------------
# 5) Bronze → Silver: customers
# -----------------------
silver_customers = (
    customers_bronze_sdf
    .withColumn("CustomerID",  F.col("cust_id"))
    .withColumn("Name",        F.col("name"))
    .withColumn("Email",       F.col("email"))
    .withColumn("CustomerName",F.col("account_name"))
    .select("CustomerID","Name","Email","CustomerName")
)

# -----------------------
# 6) Bronze → Silver: products
# -----------------------
silver_products = (
    products_bronze_sdf
    .withColumn("ProductCode", F.col("product_code"))
    .withColumn("ProductName", F.col("product_name"))
    .select("ProductCode","ProductName")
)

# -----------------------
# 7) Write Silver tables as Delta (no database, Lakehouse Tables/)
# -----------------------
(silver_invoices.write.format("delta").mode("overwrite").option("overwriteSchema","true")
 .save("Tables/Silver/Billing/invoices"))

(silver_customers.write.format("delta").mode("overwrite").option("overwriteSchema","true")
 .save("Tables/Silver/Billing/customers"))

(silver_products.write.format("delta").mode("overwrite").option("overwriteSchema","true")
 .save("Tables/Silver/Billing/products"))

print("✅ Silver tables created in Lakehouse:")
print(" - Tables/Silver/Billing/invoices")
print(" - Tables/Silver/Billing/customers")
print(" - Tables/Silver/Billing/products")

# -----------------------
# 8) Verify schemas and samples
# -----------------------
spark.read.format("delta").load("Tables/Silver/Billing/invoices").printSchema()
spark.read.format("delta").load("Tables/Silver/Billing/invoices").show(5, truncate=False)

StatementMeta(, 82028738-1c02-46ff-8984-7e507b04eb29, 5, Finished, Available, Finished)

Bronze invoices columns: ['invoice_id', 'cust_id', 'invoice_date', 'usage_period_start', 'usage_period_end', 'total_amount_raw', 'currency', 'high_cost_product_code', 'high_cost_product_price_raw', 'payment_status_raw', 'payment_method_raw', 'due_date', 'late_fee_flag_raw', 'discount_rate_raw']
Bronze customers columns: ['cust_id', 'name', 'email', 'account_name']
Bronze products columns: ['product_code', 'product_name']
✅ Silver tables created in Lakehouse:
 - Tables/Silver/Billing/invoices
 - Tables/Silver/Billing/customers
 - Tables/Silver/Billing/products
root
 |-- InvoiceID: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- UsageStart: date (nullable = true)
 |-- UsageEnd: date (nullable = true)
 |-- TotalAmount: decimal(18,2) (nullable = true)
 |-- Currency: string (nullable = true)
 |-- ProductCode: string (nullable = true)
 |-- HighCostProductPrice: decimal(18,2) (nullable = true)
 |-- PaymentStatus: string (nullabl